# Sketchpad documents

This notebook was used to set up a RAG database using FAISS. Code from this notebook was transformed into a separate script within the final pipeline. This sketchpad was an experimental playground, which has been heavily edited with parts being added, removed, and edited as the situation called for it. As such, this file does not follow proper code conventions and should not be seen as representative for the final product.

As this consists largely of code not used in the final pipeline, it lacks proper documentation, and is largely kept for archival purposes.

In [11]:
import os, json, re, faiss
from pathlib import Path
from pypdf import PdfReader
import tiktoken
import openai
import numpy as np

### Making a RAG database

using FAISS

In [ ]:
DATA_DIR = "data/spatial_genai_storage/data_RAG"
INDEX_DIR = "data/spatial_genai_storage/database_RAG"
EMB_DIM = 3072
EMBEDDER = "text-embedding-3-large"

META_PATH = Path(INDEX_DIR) / "metadata.json"
INDEX_PATH = str(Path(INDEX_DIR) / "faiss.index")

openai.api_key = os.getenv("OPENAI_API_KEY")

In [16]:
def embed_texts(texts, batch_size=1000):
    enc = tiktoken.get_encoding("cl100k_base")
    all_embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        total_tokens = sum(len(enc.encode(t)) for t in batch)
        # max batch size for text-embedding-3-large is 300K tokens
        if total_tokens > 300000:
            print(f"Batch {i//batch_size} too large ({total_tokens} tokens), skipping or reduce batch_size")
            continue
        resp = openai.embeddings.create(model=EMBEDDER, input=batch)
        all_embs.extend([d.embedding for d in resp.data])
    return all_embs

def read_pdf(path):
    reader = PdfReader(path)
    pages = []
    for p in reader.pages:
        text = p.extract_text() or ""
        text = re.sub(r'\s+', ' ', text).strip()
        if text:
            pages.append(text)
    return "\n".join(pages)

def chunk_text(text, target_tokens=300, overlap=50):
    enc = tiktoken.get_encoding("cl100k_base")
    toks = enc.encode(text)
    chunks = []
    start = 0
    while start < len(toks):
        end = start + target_tokens
        slice_toks = toks[start:end]
        chunk = enc.decode(slice_toks)
        chunks.append(chunk)
        start += target_tokens - overlap
    return chunks

def infer_metadata(filename):
    location = None
    doc_type = None
    
    loc = re.search(r'Omgevingsplan_(\w+)\.pdf', filename)
    doc = re.search(r'(\w+)_*', filename)

    if loc: location = loc.group(1)
    if doc: doc_type = doc.group(1)
    
    return {
        "source_file": filename,
        "location": location,
        "doc_type": doc_type,
        "language": "nl",
        # "polygon": "POLYGON((...))"  # TO-DO: add polygon for pre-filtering
    }

def build():
    vectors = []
    metadatas = []
    texts_flat = []
    for pdf in Path(DATA_DIR).glob("*.pdf"):
        print(f"Processing PDF: {pdf.name}")
        raw = read_pdf(pdf)
        chunks = chunk_text(raw)
        print(f"  - Extracted {len(chunks)} chunks")
        md_base = infer_metadata(pdf.name)
        for i, ch in enumerate(chunks):
            metadatas.append({**md_base, "chunk_id": f"{pdf.name}::${i}", "text": ch})
            texts_flat.append(ch)
    print("Generating embeddings...")   
    embs = embed_texts(texts_flat)
    index = faiss.IndexFlatL2(EMB_DIM)
    index.add(np.array(embs).astype('float32'))
    faiss.write_index(index, INDEX_PATH)
    with open(META_PATH, "w") as f:
        json.dump(metadatas, f, ensure_ascii=False, indent=2)
    print("Index and metadata saved.")


def search(query, k=5, location=None):
    print(f"Searching for: '{query}' (location filter: {location})")
    with open(META_PATH) as f:
        metas = json.load(f)
    index = faiss.read_index(INDEX_PATH)
    qv = embed_texts([query])[0]
    D, I = index.search(np.array([qv]).astype('float32'), min(k*4, len(metas)))
    results = []
    for dist, idx in zip(D[0], I[0]):
        md = metas[idx]
        if location and md["location"] != location:
            continue
        results.append({**md, "score": float(dist)})
        if len(results) >= k:
            break
    return results

In [17]:
build()

Processing PDF: Omgevingsplan_Amersfoort.pdf
  - Extracted 924 chunks
Processing PDF: Omgevingsplan_Baarn.pdf
  - Extracted 921 chunks
Processing PDF: Omgevingsplan_Bunnik.pdf
  - Extracted 927 chunks
Processing PDF: Omgevingsplan_Bunschoten.pdf
  - Extracted 929 chunks
Processing PDF: Omgevingsplan_DeBilt.pdf
  - Extracted 922 chunks
Processing PDF: Omgevingsplan_DeRondeVenen.pdf
  - Extracted 930 chunks
Processing PDF: Omgevingsplan_Eemnes.pdf
  - Extracted 927 chunks
Processing PDF: Omgevingsplan_Houten.pdf
  - Extracted 927 chunks
Processing PDF: Omgevingsplan_Ijsselstein.pdf
  - Extracted 928 chunks
Processing PDF: Omgevingsplan_Leusden.pdf
  - Extracted 925 chunks
Processing PDF: Omgevingsplan_Lopik.pdf
  - Extracted 925 chunks
Processing PDF: Omgevingsplan_Montfoort.pdf
  - Extracted 926 chunks
Processing PDF: Omgevingsplan_Nieuwegein.pdf
  - Extracted 1032 chunks
Processing PDF: Omgevingsplan_Oudewater.pdf
  - Extracted 924 chunks
Processing PDF: Omgevingsplan_Renswoude.pdf
  -

In [ ]:
res = search("Regels voor bouwhoogte binnen centrumgebied", k=20, location="Utrecht")
for r in res:
    print(r["score"], r["location"], r["text"][:160])

Searching for: 'Regels voor bouwhoogte binnen centrumgebied' (location filter: Utrecht)
0.7231884598731995 Utrecht alingen Bij toepassing van de regels worden onderstaande regels over het meten en berekenen gebruikt. anti-dubbeltelregelGrond die eenmaal in aanmerking is geno
0.7613204717636108 Utrecht  regels ten aanzien van de maximale bouwhoogte van gebouwen en toestaan dat de bouwhoogte van de gebouwen wordt verhoogdten behoeve van plaatselijke verhogingen
0.7628782987594604 Utrecht ende,afgewerkte maaiveld minimaal 2,2 meter bedraagt; en b. als de overschrijding voldoet aan de volgende regels: 1. de overschrijding past in het doel van de f
0.7788470983505249 Utrecht ige woningen ontstaan of als de woning hierdoor wordt omgezetnaar onzelfstandige woonruimte gelden de regels van artikel 6.7. Artikel 4.16 Beoordelingsregels vo
0.7925353646278381 Utrecht  de gemeente meestal een hoogte van maximaal 3 meter toestaan. Met deze hoogte is inpandig een stahoogte van 2,6 meter mogelijk, waar

In [21]:
res[0]['text']

'alingen Bij toepassing van de regels worden onderstaande regels over het meten en berekenen gebruikt. anti-dubbeltelregelGrond die eenmaal in aanmerking is genomen bij het toestaan van een bouwplan waaraan uitvoering is gegeven of alsnog kan worden gegeven, blijft bij debeoordeling van latere bouwplannen buiten beschouwing. dakhellingDe hoek die het dakvlak maakt ten opzichte van het horizontale vlak. hoogte- en dieptematenDe hoogte van een bouwwerk: de afstand vanaf het peil tot aan het hoogste punt van het gebouw of van een bouwwerk, geen gebouw zijnde, met uitzonderingvan ondergeschikte bouwonderdelen, zoals schoorstenen, antennes en naar de aard daarmee gelijk te stellen bouwonderdelen. De goothoogte van een bouwwerk: de afstand vanaf het peil tot aan de bovenkant van de goot, de druiplijn, het boeiboord of een daarmee gelijk te stellenconstructiedeel.De hoogte van een kap: de afstand vanaf de bovenkant van de goot, de druiplijn, het boeiboord'